# 문제 6

[Kaggle 형] train_prob.csv로 failure 예측하는 모델을 만들고, 

test_prob.csv에 대한 failure가 1일 확률 예측하여 다음과 같은 형식의 answer6.csv를 만들어라. 

측정 지표는 AUC(area under of ROC curve)이다. id 는 테스트 케이스의 id 이고, failure에는 failure가 1이 될 확률이다.

id,failure

16115, 0.1

16116, 0.2


**강사: 멀티캠퍼스 강선구(sunku0316.kang@multicampus.com, sun9sun9@gmail.com)**

In [1]:
# 실행 환경 확인

import pandas as pd
import numpy as np
import sklearn
import scipy
import statsmodels
import mlxtend
import sys
import xgboost as xgb

print(sys.version)
for i in [pd, np, sklearn, scipy, mlxtend, statsmodels, xgb]:
    print(i.__name__, i.__version__)

3.7.4 (tags/v3.7.4:e09359112e, Jul  8 2019, 20:34:20) [MSC v.1916 64 bit (AMD64)]
pandas 0.25.1
numpy 1.18.5
sklearn 0.21.3
scipy 1.5.2
mlxtend 0.15.0.0
statsmodels 0.11.1
xgboost 0.80


In [2]:
df_train = pd.read_csv('train_prob.csv', index_col = 'id')
df_test = pd.read_csv('test_prob.csv', index_col = 'id')

In [3]:
# 문제 1: na_1, na_2, measurement_3 isna, measurement_5 isna
df_train = df_train.assign(
    na_1 = lambda x: x['measurement_3'].isna(),
    na_2 = lambda x: x['measurement_5'].isna()
)
df_test = df_test.assign(
    na_1 = lambda x: x['measurement_3'].isna(),
    na_2 = lambda x: x['measurement_5'].isna()
)

In [4]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression

imp = IterativeImputer(
    estimator = LinearRegression(fit_intercept = True), 
    random_state=123
)
X_imp = ['measurement_{}'.format(i) for i in range(3, 10)] + ['measurement_17']

df_train[X_imp] = df_train.groupby('product_code')[X_imp].apply(
    lambda x: pd.DataFrame(imp.fit_transform(x), index = x.index, columns = X_imp)
)

df_test[X_imp] = df_test.groupby('product_code')[X_imp].apply(
    lambda x: pd.DataFrame(imp.fit_transform(x), index = x.index, columns = X_imp)
)

X_mean = ['measurement_{}'.format(i) for i in range(10, 17)]
df_train[X_mean] = df_train.groupby('product_code')[X_mean].transform(
    lambda x: x.fillna(x.mean())
)
df_test[X_mean] = df_test.groupby('product_code')[X_mean].transform(
    lambda x: x.fillna(x.mean())
)

In [5]:
loading_mean = pd.concat([df_train['loading'], df_test['loading']]).mean()
df_train['loading'] = df_train['loading'].fillna(loading_mean)
df_test['loading'] = df_test['loading'].fillna(loading_mean)
df_train['loading_log'] = np.log(df_train['loading'])
df_test['loading_log'] = np.log(df_test['loading'])

In [6]:
# loading_log 
# attribute_0, attribute_1 은 failure와 연관 x
# LR STD 'loading', 'measurement_1', 'measurement_4', 'measurement_14', 'measurement_17', 'na_1' 0.5838326230092876
# LDA transform / predict
# LR PCA measurement_0 ~ 17, n_components = 7 0.5808025223153578
# RF {'n_estimators': 15, 'max_depth': 7, 'min_samples_split': 512} measurement_0~ 17, na_1, na_2, loading

# Kaggle형 풀이 단계

Step 0: Kaggle용 데이터셋을 만든다.

Step 1: 검증 방법을 정하고, 검증 루틴을 만듭니다.

Step 2: Baseline 모델을 만듭니다

Step 3: 모델 선택 루틴을 만듭니다.

|id|failure|
|----|----|
|16115| 0.1|
|16116| 0.2|

....	

Step 4: 모델 개선 작업을 합니다.

In [7]:
X_all = df_test.columns.tolist()
df_train.isna().sum().sum(), df_test.isna().sum().sum()

(0, 0)

In [8]:
df_train['product_code'].unique(), df_test['product_code'].unique()

(array(['A', 'B', 'C', 'E'], dtype=object), array(['D'], dtype=object))

In [9]:
from sklearn.model_selection import GroupKFold

gkf = GroupKFold(n_splits = 4)

for train_idx, valid_idx in gkf.split(df_train[X_all], df_train['failure'], groups = df_train['product_code']):
    print(
        df_train.iloc[train_idx]['product_code'].unique(), 
        df_train.iloc[valid_idx]['product_code'].unique()
    )

['A' 'B' 'E'] ['C']
['A' 'B' 'C'] ['E']
['A' 'C' 'E'] ['B']
['B' 'C' 'E'] ['A']


In [10]:
# LR STD 'loading', 'measurement_1', 'measurement_4', 'measurement_14', 'measurement_17', 'na_1' 0.5838326230092876
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
ct = ColumnTransformer([
    ('std', StandardScaler(), ['loading', 'measurement_1', 'measurement_4', 'measurement_14', 'measurement_17']),
    ('pt', 'passthrough', ['na_1'])
])
reg_lr = make_pipeline(
    ct, LogisticRegression(solver = 'lbfgs')
)

result = cross_validate(
    reg_lr, df_train[X_all], df_train['failure'], groups = df_train['product_code'], scoring = 'roc_auc', 
    cv = gkf, return_train_score = True
)

In [11]:
reg_lr.fit(df_train[X_all], df_train['failure'])
prd = reg_lr.predict_proba(df_test[X_all])[:, 1]
pd.DataFrame({
    'id': df_test.index,
    'failure': prd
}).to_csv('answer6.csv', index = None)